1단계는 데이터 준비야.

In [8]:
from fastai.vision.all import *  #untar_data쓰기 위해서 필요해.

path = untar_data(URLs.MNIST_SAMPLE)
#전체 데이터를 다운로드해. path 에는 전체 데이터의 경로들이 있어.

In [3]:
threes = (path/'train'/'3').ls().sorted()
sevens = (path/'train'/'7').ls().sorted()
#3과 7 학습데이터의 경로들을 빼둬.

In [4]:
seven_tensors = [tensor(Image.open(o)) for o in sevens]
three_tensors = [tensor(Image.open(o)) for o in threes]
#3과 7 학습데이터 경로를 열어서 텐서 형태로 담아둬.
#리스트에는 6000개의 개별 텐서들이 담길 거야. len(seven_tensors)=6000이야.

In [5]:
stacked_sevens = torch.stack(seven_tensors).float()/255
stacked_threes = torch.stack(three_tensors).float()/255
#개별 텐서들을 묶어서 하나의 텐서로 만든 후 255로 나눠서 0~1값으로 조정해.
#stack을 하는 이유: 원래 2D텐서들이 묶여있는 ‘리스트’ 였는데, 이것을 하나의 ‘텐서’로 만드는거야. 그래서 결과적으로 3D텐서를 만드는거야. 리스트보다 텐서가 계산이 빠르니까. 모양은 같은데 리스트에서 텐서로 바뀐 거야.
#stacked_sevens.shape=[6000,28,28]이야.

In [6]:
valid_3_tens = torch.stack([tensor(Image.open(o)) for o in (path/'valid'/'3').ls()])
valid_3_tens = valid_3_tens.float()/255
valid_7_tens = torch.stack([tensor(Image.open(o)) for o in (path/'valid'/'7').ls()])
valid_7_tens = valid_7_tens.float()/255
#검증데이터에 대해서도 동일한 작업을 해.
#valid_7_tens.shape=[1000,28,28]이야.

In [7]:
train_x = torch.cat([stacked_threes, stacked_sevens]).view(-1, 28*28)
train_y = tensor([1]*len(threes) + [0]*len(sevens)).unsqueeze(1)
dset = list(zip(train_x, train_y))
#입력데이터를 묶어주는 거야. 3과 7텐서를 묶어준 후 2D텐서로 형태를 바꿔줘.
#train_x.shape=[1100,2828]이야.
#정답데이터를 만들어줘. 입력과 동일하게 2D텐서 형태여야 해.
#train_y.shape=[1100,1]이야.
#그리고나서 입력과 정답데이터를 튜플 형태로 묶어준 것이 dset이야.

In [9]:
valid_x = torch.cat([valid_3_tens, valid_7_tens]).view(-1, 28*28)
valid_y = tensor ([1]*len(valid_3_tens) + [0]*len(valid_7_tens)).unsqueeze(1)
valid_dset = list(zip(valid_x, valid_y))
#검증데이터에 대해서도 동일한 작업을 해.

#len(dset), len(valid_dset) = (12396, 2038)

In [10]:
dl = DataLoader(dset, batch_size=256)
valid_dl = DataLoader(valid_dset, batch_size=256)
#미니배치로 만들어 학습해야 효율적이야.
#데이터셋에서 256개의 튜플을 무작위로 골라서 미니배치로 만들어.

#len(dl),len(valid_dl) = (49, 8)

2단계는 각종 함수를 만드는 거야.
- 학습할 때 필요한 함수를 만들어.
- 가중치추출함수, 모델 함수, 손실함수를 만들어.

In [12]:
def init_params(size, std=1.0):
    return (torch.randn(size)*std).requires_grad_()
#가중치를 추출하는 함수야. 이걸 이용해서 가중치와 편향을 선정할거야.

In [13]:
def linear1(xb):
    return xb@weights + bias
#모델 함수야. 입력데이터에 가중치를 곱하고 편향을 더한 값을 예측으로 리턴할거야.

In [14]:
def mnist_loss(predictions, targets):
    predictions = predictions.sigmoid()
    return torch.where(targets==1, 1-predictions, predictions).mean()
#손실 함수야. 예측값을 우선 0~1값으로 변형해줘.
#예측값이 0.7이고 타겟이 1이라면 0.3의 손실값을 리턴하거야.
#예측값이 0.7이고 타겟이 0이라면 0.7의 손실값을 리턴하거야.

3단계는 한 에포크 당 진행할 학습을 담은 함수를 만드는 거야.  
1. 우선 파라미터의 기울기를 계산하고,  
2. 파라미터에서 기울기와 학습률을 곱한 만큼을 빼주면서 파라미터를 업데이트 해야해.

- x축이 파라미터를 나타내고 손실함수가 2차함수라고 가정하자. 국소 최솟값은 x=1일 때야.
- 지금 파라미터x값이 2라면, 손실이 크고, 또 손실에 대한 기울기가 큰 상황이야.
- 업데이트를 거치면서 x값이 점점 작아지고, 또 기울기도 점점 줄어들도록 만들어.
- 결국 기울기가 작은 지점, 즉 손실 값이 가장 작아질 때까지 파라미터를 갱신하는 게 학습이야.

In [22]:
def calc_grad(xb, yb, model):
    preds = model(xb)
    loss = mnist_loss(preds, yb)
    loss.backward()

In [23]:
def train_epoch(model, Ir, params):
    for xb,yb in dl:
        calc_grad(xb, yb, model)
        for p in params:
            p.data -= p.grad*lr
            p.grad.zero_()
            
#미니배치를 가지고 예측한 후, 손실에 대한 파라미터의 기울기를 구해.
#그 기울기를 이용해서 파라미터를 업데이트해.
#어차피 다음 미니배치를 가지고 calc_grad를 수행할 때 loss.backward로 기울기를 새로 구할 텐데 왜 초기화 해?
#loss.backward는 기울기를 새로 구하는 게 아니라
#방금전 앞서 계산된 기울기에 더하기 때문에 이전 기울기를 0으로 설정해줘야 해.

4단계는 정확도를 평가하는 함수를 만드는 거야.

In [24]:
def batch_accuracy(xb, yb):
    preds = xb.sigmoid()
    correct = (preds>0.5) == yb
    return correct.float().mean()
#한 배치 내에서 정답이면 1, 오답이면 0을 누적한 후 평균값을 리턴해. (0.7과 같은 숫자로 나와)

#float() 하는 이유: 1, 0과 같은 정수는 mean()메서드를 쓸 수 없어.

In [25]:
def validate_epoch(model):
    accs = [batch_accuracy(model(xb), yb) for xb, yb in valid_dl]
    return round(torch.stack(accs).mean().item(), 4)
#검증데이터의 모든 배치의 정확도를 구한 후 평균값을 리턴해.

5단계는 학습을 진행하는 거야.

In [26]:
weights = init_params((28*28, 1))
bias = init_params(1)
params = weights, bias
lr = 1.
#파라미터를 추출하고, 학습률을 지정해.

In [27]:
for i in range(20):
    train_epoch(linear1, lr, params)
    print(validate_epoch(linear1), end=' ')
#20회 에포크로 학습을 해. 즉, 전체 데이터셋을 20번 돌리며 학습해.
#1에포크가 끝날 때마다 정확도를 출력해.

0.6117 0.8427 0.9198 0.9457 0.9555 0.9603 0.9638 0.9657 0.9677 0.9686 0.9677 0.9681 0.9681 0.9696 0.9696 0.9706 0.9711 0.9711 0.9711 0.9711 

6단계는 옵티마이저 만들기야.
- 옵티마이저 클래스에 학습률과 파라미터를 저장해.
- 이건 파라미터를 “최적화”하는 클래스이기 때문에
1. 파라미터를 업데이트하는 함수랑
2. 파라미터의 기울기 초기화하는 함수가 담겨.
- 옵티마이저 클래스를 정의해두면 학습 루프를 간단하게 짤 수 있어.

In [28]:
linear_model = nn.Linear(28*28, 1)
w,b = linear_model.parameters()

#nn.Linear는 모듈이야.
#init_params(파라미터 추출) 과 linear1(모델) 작동을 하나로 수행해.
#이 클래스 내에 파라미터가 저장돼.
#파라미터를 보려면 linear_model.parameters()하면 돼.

In [32]:
class BasicOptim:
    def __init__(self, params, lr):
        self.params, self.lr = list(params), lr
    
    #파라미터를 업데이트하는 함수야.
    def step(self, *args, **kwargs):
        for p in self.params: p.data -= p.grad.data * self.lr
    
    #파라미터의 기울기를 초기화하는 함수야.
    def zero_grad(self, *args,**kwargs):
        for p in self.params: p.grad = None

In [33]:
opt = BasicOptim(linear_model.parameters(), lr)

In [35]:
def train_epoch(model) :
    for xb,yb in dl:
        calc_grad(xb, yb, model)
        opt.step()
        opt.zero_grad()
        
def train_model(model, epochs) :
    for i in range(epochs) :
        train_epoch (model)
        print (validate_epoch (model), end='')

train_model(linear_model, 20)

0.49320.87840.81980.90820.93260.94530.95510.96340.96580.96680.96870.97070.97310.97510.97560.97650.97750.9780.97850.9785

7단계는 fastai가 제공하는 옵티마이저를 사용하는 거야.
- SGD는 BasicOptim과 동일하게 작동해.

In [36]:
linear_model = nn. Linear(28*28,1)
opt = SGD(linear_model.parameters(), lr)
train_model(linear_model, 20)

0.49320.88280.8130.90970.93160.94530.95550.96140.96530.96730.96920.97120.97360.97510.97610.97650.97750.9780.97850.9785

8단계는 train_model 함수 대신 Learner.fit 으로 학습 시키는 거야.
- Learner.fit 을 사용하려면 Learner를 생성해야하고
- Learner를 생성하려면 dls를 생성해야해.

In [37]:
dls = DataLoaders(dl, valid_dl)
learn = Learner(dls, nn.Linear(28*28, 1), opt_func=SGD, loss_func=mnist_loss, metrics=batch_accuracy)
learn.fit(10, lr=lr)

epoch,train_loss,valid_loss,batch_accuracy,time
0,0.636456,0.503230,0.495584,00:00
1,0.456453,0.225553,0.801276,00:00
2,0.170107,0.163060,0.855250,00:01
3,0.075872,0.099381,0.916094,00:00
4,0.041120,0.074095,0.935231,00:00
5,0.027521,0.060078,0.949951,00:00
6,0.021912,0.051233,0.956330,00:00
7,0.019393,0.045280,0.963199,00:00
8,0.018095,0.041053,0.966143,00:00
9,0.017301,0.037912,0.967125,00:00


9단계는 모델에 비선형성을 추가하는 거야.
- nn.ReLU() 함수가 비선형 모델이야.

In [38]:
simple_net = nn.Sequential(
    nn.Linear(28*28, 30),
    nn.ReLU(),
    nn.Linear(30,1))

learn = Learner(dls, simple_net, opt_func=SGD, loss_func=mnist_loss, metrics=batch_accuracy)
learn.fit(40, 0.1)

epoch,train_loss,valid_loss,batch_accuracy,time
0,0.289257,0.413602,0.505397,00:01
1,0.138044,0.220409,0.812071,00:01
2,0.077594,0.112413,0.917566,00:00
3,0.051764,0.076591,0.941609,00:00
4,0.039683,0.060020,0.956820,00:01
5,0.033456,0.050649,0.964671,00:01
6,0.029839,0.044723,0.967615,00:00
7,0.027465,0.040660,0.968106,00:00
8,0.025737,0.037696,0.969578,00:01
9,0.024388,0.035432,0.971050,00:00


10단계는 18계층으로 구성된 모델을 학습시키는 거야.
- 계층이 아주 깊고, 거의100%에 가까운 정확도를 얻을 수 있어.

In [40]:
dls = ImageDataLoaders.from_folder(path)
learn = cnn_learner(dls, resnet18, pretrained=False, loss_func=F.cross_entropy, metrics=accuracy)
learn.fit_one_cycle(1, 0.1)

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


epoch,train_loss,valid_loss,accuracy,time
0,0.064714,0.007904,0.996565,00:07
